# Fake/Real news dataset
### Завантажити датасет можна за наступним посиланням:  
> https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset/download

### Загалом датасет має наступний вигляд: 

```Index(['title', 'text', 'subject', 'date'], dtype='object')```

Для подальшої обробки було додано колонку 'Authenticity' та було об'єднано датасети в один news_data.






In [ ]:
import os
import pandas as pd

real = pd.read_csv("True.csv")
fake = pd.read_csv("Fake.csv")

print(real.columns)
fake['Authenticity'] = 0
real['Authenticity'] = 1
print(real.columns)

news_data = fake.append(real)

Index(['title', 'text', 'subject', 'date'], dtype='object')
Index(['title', 'text', 'subject', 'date', 'Authenticity'], dtype='object')


### Далі конветуємо колекцію нашого тексту у матрицю токенів

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
import pandas as pd
import numpy as np

valid_data = shuffle(news_data)

countvec = CountVectorizer()
count_dat = countvec.fit_transform(valid_data['text'].astype('str'))

###Ділимо датасет на тренувальний та валідаційний

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(count_dat, valid_data['Authenticity'], test_size=0.3)

###Логістична регресія для CountVectorizer виглядає наступним чином

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(solver="liblinear")
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print("Logistic regression accuracy: ", accuracy)

Logistic regression accuracy:  0.9960653303637713


###Наївний Баєс для CountVectorizer виглядає наступним чином

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_valid)

accuracy = accuracy_score(y_valid, y_pred)

print("NB accurracy: ", accuracy)

NB accurracy:  0.9529324424647364


###SVC для CountVectorizer виглядає наступним чином

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC(max_iter=100000)
svm.fit(X_train, y_train)
svm_acc = svm.score(X_valid, y_valid)
print("SVM accuracy: ", svm_acc)

SVM accuracy:  0.9956941351150705


###Далі пробуємо замість CountVectorizer використати TfidfVectorizer для токенізації нашого текту

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
import pandas as pd
import numpy as np

valid_data = shuffle(news_data)

tfidf = TfidfVectorizer()
X_text = tfidf.fit_transform(valid_data['text'].astype('str'))

###Знову ділимо датасет на тренувальні та валідаційні дані

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_text, valid_data['Authenticity'], test_size=0.3)

###Логістична регресія для TfidfVectorizer виглядає наступним чином

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(solver="liblinear")
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print("Logistic regression accuracy: ", accuracy)

Logistic regression accuracy:  0.9868596881959911


###Далі представлена логістична регресія з динамічним підбором гіперпараметрів

Наступна стрічка відповідає за перебір гіперпараметрів від (0.001 до 10^10)
> np.logspace(-3, 10, 24) 



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
  
c_space = np.logspace(-3, 10, 24)

param_grid = {
    'C': c_space
    }
  
logreg_cv = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv = 5)
  
logreg_cv.fit(X_train, y_train)
  
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameters: {'C': 14924955.45051826}
Best score is 0.9665251436600079


###Наївний Баєс для TfidfVectorizer виглядає наступним чином

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_valid)

accuracy = accuracy_score(y_valid, y_pred)

print("NB accurracy: ", accuracy)

NB accurracy:  0.9364513734224202


###SVC для TfidfVectorizer виглядає наступним чином

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC(max_iter=10000)
svm.fit(X_train, y_train)
svm_acc = svm.score(X_valid, y_valid)
print("SVM accuracy: ", svm_acc)

SVM accuracy:  0.9933927245731254


### Токенізація, яка повертає список оброблених токенів

In [ ]:
import spacy
from typing import List


spacy_nlp = spacy.load("en_core_web_sm")


def tokenize(text: str) -> List[str]:

  disable = ["ner", "parser", "tagger", "lemmatizer", "tok2vec", "senter", "attribute_ruler"]
  tokens = spacy_nlp(text, disable=disable)
  return [str(token).lower() for token in tokens]

### Glove
Завантажуємо 50к найбільш уживаних слів з Glove.

In [ ]:
import gensim.downloader
from gensim.models import KeyedVectors
import tqdm

def load_glove_subset(max_n: int) -> KeyedVectors:
    all_glove = gensim.downloader.load("glove-wiki-gigaword-200")
    subset = KeyedVectors(all_glove.vector_size)
    for word in tqdm.tqdm(all_glove.vocab, total=max_n):
        if len(subset.vectors) >= max_n:
            break
        subset.add(word, all_glove[word])
    return subset

glove = KeyedVectors.load("http://134.209.248.229:8081/glove-50k.bin")

###Функція bag_of_embeddings використовує раніше написаний токенізатор та перебирає слова з датасету glove. Оскільки тексти є різних розмірів, тому ми використовуємо лише медіанне значення для кожного тексту.

In [ ]:
import numpy as np

def bag_of_embeddings(dataset):
    X = []
    for index, doc in dataset.iterrows():
        tokens = tokenize(doc['text'])
        token_vectors = []
        for token in tokens:
            if token in glove:
                token_vectors.append(glove[token])
                
        if token_vectors:
          doc_vector = np.array(token_vectors).mean(axis=0)
        if not isinstance(doc_vector, float):
          X.append(doc_vector)
        else :
          dataset.drop(dataset.index[index])

    X = np.stack(X)
    y = np.array(dataset['Authenticity'])
    return (X, y)

###Далі розділяємо наш датасет на тренувальні та валідаційні дані та застосовуємо bag_of_embeddings() для токенізації.

In [ ]:
from sklearn.utils import shuffle
import pandas as pd

valid_data = shuffle(news_data)

validation = []
training = []
for i in range(len(valid_data)):
  if i >= 2000:
    training.append(valid_data.iloc[i])
  else:
    validation.append(valid_data.iloc[i])

valid_data = pd.DataFrame (validation, columns=['title', 'text', 'subject', 'date', 'Authenticity'])
train_data = pd.DataFrame (training, columns=['title', 'text', 'subject', 'date', 'Authenticity'])

#for index, item in train_data.iterrows():
#  print(type(item["text"]))
print(len(valid_data))
print(len(train_data))
X_train, y_train = bag_of_embeddings(train_data)
print(len(valid_data))
print(len(train_data))
X_valid, y_valid = bag_of_embeddings(valid_data)

2000
42898
2000
42898


###Логістична регресія для bag_of_embeddings виглядає наступним чином

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(solver="liblinear")
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print("Logistic regression accuracy: ", accuracy)

Logistic regression accuracy:  0.9585


###Наївний Баєс для bag_of_embeddings виглядає наступним чином

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_valid)

accuracy = accuracy_score(y_valid, y_pred)

print("NB accurracy: ", accuracy)

NB accurracy:  0.8715


###SVC для bag_of_embeddings виглядає наступним чином

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC(max_iter=10000)
svm.fit(X_train, y_train)
svm_acc = svm.score(X_valid, y_valid)
print("SVM accuracy: ", svm_acc)

SVM accuracy:  0.963
